In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), ".."))
sys.path.append("../../wrench/")

In [2]:
from rulecleaner_src.lfs_tree import keyword_labelling_func_builder
from rulecleaner_src.TreeRules import SPAM, HAM, ABSTAIN, PredicateNode
from rulecleaner_src.LFRepair import populate_violations, fix_rules_with_solver_input
from rulecleaner_src.utils import run_label_model_with_funcs, select_user_input, clean_text
import re
import psycopg2
import pandas as pd
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
import pulp
from sklearn.metrics import accuracy_score, classification_report
import random
from collections import deque, defaultdict
import numpy as np
import pickle
from IPython.display import Image, display 
import datetime
import itertools

[nltk_data] Downloading package words to /Users/chenjieli/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenjieli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from rulecleaner_src.example_tree_rules import (
gen_amazon_funcs,
gen_professor_teacher_funcs,
gen_painter_architecht_funcs,
gen_imdb_funcs,
gen_pj_funcs,
gen_pp_funcs,
gen_yelp_funcs,
gen_plots_funcs,
gen_fakenews_funcs,
gen_dbpedia_funcs,
gen_agnews_funcs,
gen_tweets_funcs,
gen_spam_funcs
)


In [4]:
dataset_dict = {
    "plots": gen_plots_funcs,
    "amazon": gen_amazon_funcs,
    "dbpedia": gen_dbpedia_funcs,
    "agnews": gen_agnews_funcs,
    "physician_professor": gen_pp_funcs,
    "imdb": gen_imdb_funcs,
    "fakenews": gen_fakenews_funcs,
    "yelp": gen_yelp_funcs,
    "photographer_journalist": gen_pj_funcs,
    "professor_teacher": gen_professor_teacher_funcs,
    "painter_architect": gen_painter_architecht_funcs,
    "tweets": gen_tweets_funcs,
    "spam": gen_spam_funcs,
}



In [5]:
from rulecleaner_src.main import main

In [ ]:

user_input_sizes = [40]
# random_states = [1,2,3,4,5,6,7,8, 9, 10]
random_states = [1]
lf_acc_threshs = [0.7]
instance_acc_threshs = [0.8]
non_abstain_threshs = [0.8]
datasets = list(dataset_dict)
func_dictionary = [dataset_dict]
instance_acc_on_valids=[False]
use_non_abstains=[False]
model_types = ['snorkel']
pfile_name_prefix = ['res_folder/']
num_class=[2]



In [7]:
input_params = list(itertools.product(
    user_input_sizes,
    lf_acc_threshs,
    instance_acc_threshs,
    non_abstain_threshs,
    datasets,
    random_states,
    func_dictionary,
    instance_acc_on_valids,
    use_non_abstains,
    pfile_name_prefix,
    ['witan'],
    num_class,
    model_types
))


In [8]:
input_params

[(40,
  0.7,
  0.8,
  0.8,
  'plots',
  1,
  {'plots': <function rulecleaner_src.example_tree_rules.gen_plots_funcs()>,
   'amazon': <function rulecleaner_src.example_tree_rules.gen_amazon_funcs()>,
   'dbpedia': <function rulecleaner_src.example_tree_rules.gen_dbpedia_funcs()>,
   'agnews': <function rulecleaner_src.example_tree_rules.gen_agnews_funcs()>,
   'physician_professor': <function rulecleaner_src.example_tree_rules.gen_pp_funcs()>,
   'imdb': <function rulecleaner_src.example_tree_rules.gen_imdb_funcs()>,
   'fakenews': <function rulecleaner_src.example_tree_rules.gen_fakenews_funcs()>,
   'yelp': <function rulecleaner_src.example_tree_rules.gen_yelp_funcs()>,
   'photographer_journalist': <function rulecleaner_src.example_tree_rules.gen_pj_funcs()>,
   'professor_teacher': <function rulecleaner_src.example_tree_rules.gen_professor_teacher_funcs()>,
   'painter_architect': <function rulecleaner_src.example_tree_rules.gen_painter_architecht_funcs()>,
   'tweets': <function ru

In [9]:
for ip in input_params:
    main(*ip)

/Users/chenjieli/Desktop/RuleCleaner/rulecleaner_src/../rulecleaner_src/utils.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df = pd.read_sql(f'SELECT * FROM {dataset_name}', conn)
2025-06-03 21:17:05,711 - root - INFO - Computing O...
2025-06-03 21:17:05,714 - root - INFO - Estimating \mu...


user_input_size = 40
lf_acc_thresh = 0.7
instance_acc_thresh = 0.8
min_non_abstain_thresh = 0.8
dataset_name = plots
random_state = 1
funcs_dictionary = {'plots': <function gen_plots_funcs at 0x1296180d0>, 'amazon': <function gen_amazon_funcs at 0x129603c70>, 'dbpedia': <function gen_dbpedia_funcs at 0x1296181f0>, 'agnews': <function gen_agnews_funcs at 0x129618280>, 'physician_professor': <function gen_pp_funcs at 0x129603f40>, 'imdb': <function gen_imdb_funcs at 0x129603e20>, 'fakenews': <function gen_fakenews_funcs at 0x129618160>, 'yelp': <function gen_yelp_funcs at 0x129618040>, 'photographer_journalist': <function gen_pj_funcs at 0x129603eb0>, 'professor_teacher': <function gen_professor_teacher_funcs at 0x129603d00>, 'painter_architect': <function gen_painter_architecht_funcs at 0x129603d90>, 'tweets': <function gen_tweets_funcs at 0x129618310>, 'spam': <function gen_spam_funcs at 0x1296183a0>}
instance_acc_on_valid = False
use_non_abstain = False
pickle_result_file_name_prefix 

2025-06-03 21:17:06,234 - root - INFO - [0 epochs]: TRAIN:[loss=0.105]                                                                         | 0/500 [00:00<?, ?epoch/s]
2025-06-03 21:17:06,258 - root - INFO - [100 epochs]: TRAIN:[loss=0.000]
2025-06-03 21:17:06,281 - root - INFO - [200 epochs]: TRAIN:[loss=0.000]
2025-06-03 21:17:06,303 - root - INFO - [300 epochs]: TRAIN:[loss=0.000]
2025-06-03 21:17:06,326 - root - INFO - [400 epochs]: TRAIN:[loss=0.000]███████████████▍                                           | 327/500 [00:00<00:00, 3267.74epoch/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 3601.50epoch/s]
2025-06-03 21:17:06,349 - root - INFO - Finished Training
2025-06-03 21:17:06,440 - pulp.apis.core - DEBUG - /Users/chenjieli/.pyenv/versions/3.10.0/envs/rulecleaner/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sl/1_4_3jg90p50ft92b5k6mgv0000


        Out of 1937 sentences, 1461 got signal for prediction.
        390 predictions were incorrect.
        Accuracy on valid = 0.7331
        Overall accuracy = 0.5529
    
lf_acc: 0.7, ins_acc:0.8, min_non_abstain_thresh:0.8
labeling_functions: ['lf_1', 'lf_2', 'lf_3', 'lf_4', 'lf_5', 'lf_6', 'lf_7', 'lf_8', 'lf_9', 'lf_10']
num_instances: 40


2025-06-03 21:17:06,974 - rulecleaner_src.main - DEBUG - solver output :   lf_1 lf_2 lf_3 lf_4 lf_5 lf_6 lf_7 lf_8 lf_9 lf_10
0     0    0    0    0    0    0    0   -1   -1     0
1     1    1   -1    1    1    1    1   -1    1     1
2     0    0   -1   -1    0    0    0    0    0     0
3     0    0    0    0    0    0    0   -1   -1     0
4     1    1   -1    1    1    1    1    1    1    -1
5    -1    1    1    1    1    1    1    1    1    -1
6     0    0    0    0    0    0   -1    0    0    -1
7     0    0   -1    0    0   -1    0    0    0     0
8     0    0    0    0   -1    0    0   -1    0     0
9     0   -1    0    0    0    0    0    0   -1     0
10    1    1   -1    1    1    1    1   -1    1     1
11    1    1   -1    1    1    1    1    1    1    -1
12    0   -1    0    0    0   -1    0    0    0     0
13    0    0   -1    0    0    0   -1    0    0     0
14   -1    1    1    1    1    1   -1    1    1     1
15    1   -1    1    1    1    1    1   -1    1     1
16    1   

...

pssible_labels: [-1, 0, 1]
tree_rule: rid:13PredicateNode(id=1, pred=keyword_predicate-word-(fabric,return,money,poor,garbage,poorly,terrible,useless,horrible,returning,flimsy))
    LabelNode(id=2, label=-1)
    LabelNode(id=3, label=1)
pssible_labels: [-1, 0, 1]
tree_rule: rid:13PredicateNode(id=1, pred=keyword_predicate-word-(fabric,return,money,poor,garbage,poorly,terrible,useless,horrible,returning,flimsy))
    PredicateNode(id=4, pred=keyword_predicate-word-(one), added=True)
        LabelNode(id=5, label=0, added=True)
        LabelNode(id=6, label=1, added=True)
    LabelNode(id=3, label=1)
pssible_labels: [-1, 0, 1]
tree_rule: rid:13PredicateNode(id=1, pred=keyword_predicate-word-(fabric,return,money,poor,garbage,poorly,terrible,useless,horrible,returning,flimsy))
    PredicateNode(id=4, pred=keyword_predicate-word-(one), added=True)
        LabelNode(id=5, label=0, added=True)
        LabelNode(id=6, label=1, added=True)
    PredicateNode(id=7, pred=keyword_predicate-word-(giv

2025-06-03 21:18:11,508 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 5
2025-06-03 21:18:11,534 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 7
2025-06-03 21:18:11,572 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 9
2025-06-03 21:18:11,592 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 11
2025-06-03 21:18:11,611 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 13
2025-06-03 21:18:11,614 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 15
2025-06-03 21:18:11,616 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 17
2025-06-03 21:18:11,623 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 19
2025-06-03 21:18:11,637 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 21
2025-06-03 21:18:11,638 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule size is 23
2025-06-03 21:18:11,638 - rulecleaner_src.LFRepair - DEBUG - after fix, treerule si

pssible_labels: [-1, 0, 1]
tree_rule: rid:14PredicateNode(id=1, pred=keyword_predicate-word-(pants,looks,toy,color,camera,water,phone,bag,worked,arrived,lasted))
    LabelNode(id=2, label=-1)
    LabelNode(id=3, label=1)
pssible_labels: [-1, 0, 1]
tree_rule: rid:14PredicateNode(id=1, pred=keyword_predicate-word-(pants,looks,toy,color,camera,water,phone,bag,worked,arrived,lasted))
    PredicateNode(id=4, pred=keyword_predicate-word-(one), added=True)
        LabelNode(id=5, label=0, added=True)
        LabelNode(id=6, label=1, added=True)
    LabelNode(id=3, label=1)
pssible_labels: [-1, 0, 1]
tree_rule: rid:14PredicateNode(id=1, pred=keyword_predicate-word-(pants,looks,toy,color,camera,water,phone,bag,worked,arrived,lasted))
    PredicateNode(id=4, pred=keyword_predicate-word-(one), added=True)
        LabelNode(id=5, label=0, added=True)
        LabelNode(id=6, label=1, added=True)
    PredicateNode(id=7, pred=keyword_predicate-word-(find), added=True)
        LabelNode(id=8, label=1,

/Users/chenjieli/Desktop/RuleCleaner/rulecleaner_src/../rulecleaner_src/utils.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df = pd.read_sql(f'SELECT * FROM {dataset_name}', conn)


KeyboardInterrupt: 